In [2]:
import os
import time
import socket
import tkinter
import smtplib
from socket import *
from tkinter import *
from tkinter import messagebox
from email.mime.text import MIMEText
from tkinter.messagebox import showerror

root = tkinter.Tk()
root.resizable(False, False)
root.title('SMTP EMAIL CLIENT')
ws = root.winfo_screenwidth()
hs = root.winfo_screenheight()
w = 992
h = 550
x = (ws/2) - (w/2)
y = (hs/2) - (h/2)
root.geometry('%dx%d+%d+%d' % (w, h, x, y))

def send_message():
    if user_name.get() and pass_word.get() and to_addr.get() and subject.get() and message.get():
        sender = user_name.get() + '@dcnproject.com'
        targets = to_addr.get().split(',')

        msg = MIMEText(message.get())
        msg['Subject'] = subject.get()
        msg['From'] = sender
        msg['To'] = ','.join(targets)
        msg['username'] = user_name.get()
        msg['password'] = pass_word.get()

        server = smtplib.SMTP('localhost',25)
        server.sendmail(sender, targets, msg.as_string())
        server.quit()
        to_input.delete(0,END)
        subject_input.delete(0,END)
        message_input.delete(0,END)
    else:
        showerror('Error','All feilds are required')
    
def selection1_email(event):
    to_input.delete(0,END)
    message_input.delete(0,END)
    subject_input.delete(0,END)
    file_name = sent_Emails_ListBox.get(sent_Emails_ListBox.curselection())[0]
    email = open(file_name,'r')
    mess = email.read().splitlines()
    subj = (mess[0].replace('Subject:','')).strip()
    to = (mess[2].replace('To:','')).strip()
    content = mess[-1]
    to_addr.set(to)
    subject.set(subj)
    message.set(content)

def selection2_email(event):
    to_input.delete(0,END)
    message_input.delete(0,END)
    subject_input.delete(0,END)
    file_name = rec_Emails_ListBox.get(rec_Emails_ListBox.curselection())[0]
    email = open(file_name,'r')
    mess = email.read().splitlines()
    subj = (mess[0].replace('Subject:','')).strip()
    fr = (mess[1].replace('From:','')).strip()
    content = mess[-1]
    to_addr.set(fr)
    subject.set(subj)
    message.set(content)

def receive():
    if user_name.get() and pass_word.get():
        try:
            CHUNKSIZE = 1_000_000

            sock = socket()
            sock.connect(('localhost',5000))
            sock.send(f'{user_name.get()}|{pass_word.get()}'.encode())
            with sock,sock.makefile('rb') as clientfile:
                while True:
                    raw = clientfile.readline()
                    if not raw: 
                        # no more files, server closed connection.
                        break 

                    filename = raw.strip().decode()
                    length = int(clientfile.readline())

                    path = os.path.join('',filename)
                    os.makedirs(os.path.dirname(path),exist_ok=True)

                    # Read the data in chunks so it can handle large files.
                    with open(path,'wb') as f:
                        while length:
                            chunk = min(length,CHUNKSIZE)
                            data = clientfile.read(chunk)
                            if not data: 
                                break
                            f.write(data)
                            length -= len(data)
                        else: # only runs if while doesn't break and length==0
                            continue

                    # socket was closed early.
                    break
            populateListBox1(user_name.get())
            populateListBox2(user_name.get())
        except:
            showerror('Error','Invalid Login Credentials')
    else:
        showerror('Error','Enter both username and password')
        
        
def populateListBox1(user):
    sent_Emails_ListBox.delete(0,END)
    if os.path.isdir(f'{user}/sent'):
        for email in os.listdir(f'{user}/sent'):
            sent_Emails_ListBox.insert(END,[f'{user}/sent/{email}'])
            
def populateListBox2(user):
    rec_Emails_ListBox.delete(0,END)
    if os.path.isdir(f'{user}/received'):
        for email in os.listdir(f'{user}/received'):
            rec_Emails_ListBox.insert(END,[f'{user}/received/{email}'])
    
top_frame = Frame(root, width=992, height=100, bg='#0f4c81')
top_frame.pack(side=TOP)
top_frame.pack_propagate(0)

heading_label = Label(top_frame, width=1000, height=100, bg='#0f4c81', fg='#fff',text='SMTP EMAIL CLIENT', font=('Arial',24))
heading_label.pack(side=TOP)
heading_label.config(anchor=CENTER)

left_frame = Frame(root, width=720, height=450, bg='#eee')
left_frame.pack(side=LEFT)
left_frame.pack_propagate(0)

sub_heading_label = Label(left_frame, bg='#eee', fg='#1b262c',text='EMAILS RECEIVED', font=('Arial',16))
sub_heading_label.pack(side=TOP, pady=(20,0))
sub_heading_label.config(anchor=CENTER)

rec_Emails_ListBox = Listbox(left_frame, width=720, height=7, font=('Arial',12))
rec_Emails_ListBox.pack(side=TOP, padx=(20), pady=(15,0))
rec_Emails_ListBox.bind('<<ListboxSelect>>', selection2_email)

sub_heading_label = Label(left_frame, bg='#eee', fg='#1b262c',text='EMAILS SENT', font=('Arial',16))
sub_heading_label.pack(side=TOP, pady=(20,0))
sub_heading_label.config(anchor=CENTER)

sent_Emails_ListBox = Listbox(left_frame, width=720, height=7, font=('Arial',12))
sent_Emails_ListBox.pack(side=TOP, padx=(20), pady=(15,0))
sent_Emails_ListBox.bind('<<ListboxSelect>>', selection1_email)

right_frame = Frame(root, width=272, height=450, bg='#eee')
right_frame.pack(side=RIGHT)
right_frame.pack_propagate(0)

#To
to_label = Label(right_frame, font=('Arial',12), text='TO: / FROM:', bg='#eee',fg='#1b262c')
to_label.pack(side=TOP, pady=(5,0), ipady=(5), ipadx=(75))
to_label.config(anchor='w')

to_addr = StringVar()

to_input = Entry(right_frame, font=('Arial',10), textvariable=to_addr)
to_input.pack(side=TOP, pady=(1,0), ipady=(3), ipadx=(50))

#Subject
subject_label = Label(right_frame, font=('Arial',12), text='SUBJECT:', bg='#eee',fg='#1b262c')
subject_label.pack(side=TOP, pady=(5,0), ipady=(5), ipadx=(80))
subject_label.config(anchor='w')

subject = StringVar()

subject_input = Entry(right_frame, font=('Arial',10), textvariable=subject)
subject_input.pack(side=TOP, pady=(1,0), ipady=(3), ipadx=(50))

#Message
message_label = Label(right_frame, font=('Arial',12), text='MESSAGE:', bg='#eee',fg='#1b262c')
message_label.pack(side=TOP, pady=(5,0), ipady=(5), ipadx=(75))
message_label.config(anchor='w')

message = StringVar()

message_input = Entry(right_frame, font=('Arial',10), textvariable=message)
message_input.pack(side=TOP, pady=(1,0), ipady=(3), ipadx=(50))

#Username
user_label = Label(right_frame, font=('Arial',12), text='USERNAME:', bg='#eee',fg='#1b262c')
user_label.pack(side=TOP, pady=(5,0), ipady=(5), ipadx=(70))
user_label.config(anchor='w')

user_name = StringVar()

user_input = Entry(right_frame, font=('Arial',10), textvariable=user_name)
user_input.pack(side=TOP, pady=(1,0), ipady=(3), ipadx=(50))


#Password
pass_label = Label(right_frame, font=('Arial',12), text='PASSWORD:', bg='#eee',fg='#1b262c')
pass_label.pack(side=TOP, pady=(5,0), ipady=(5), ipadx=(70))
pass_label.config(anchor='w')

pass_word = StringVar()

pass_input = Entry(right_frame, font=('Arial',10), textvariable=pass_word, show='*')
pass_input.pack(side=TOP, pady=(1,0), ipady=(3), ipadx=(50))

send_btn = Button(right_frame, font=('Arial',10), text='SEND', bg='#1b262c', fg='#fff', command=send_message)
send_btn.pack(side=TOP, fill=X, padx=(15), pady=(10,0), ipady=(5))

rec_btn = Button(right_frame, font=('Arial',10), text='FETCH', bg='#1b262c', fg='#fff', command=receive)
rec_btn.pack(side=TOP, fill=X, padx=(15), pady=(10,0), ipady=(5))


root.mainloop()